In [ ]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

# import the necessary packages
%reload_ext autoreload
%autoreload 2

# import packages
import os
import requests
import tarfile
import sqlalchemy as sa
from pathlib import Path
import pandas as pd
import plotly.express as px
import numpy as np
import pudl.analysis.allocate_net_gen as allocate_gen_fuel

import src.data_cleaning as data_cleaning
import src.gross_to_net_generation as gross_to_net_generation
import src.load_data as load_data
import src.eia930 as eia930
#import src.distribute_eia923 as distribute_eia923

In [ ]:
# specify the year for which you want data

year = 2020

## Download the data

In [ ]:
############### PUDL Database ######################

load_data.download_pudl_data(zenodo_url = 'https://zenodo.org/record/6349861/files/pudl-v0.6.0-2022-03-12.tgz')

################# eGRID data #########################

# the 2019 and 2020 data appear to be hosted on different urls
egrid_files_to_download = ['https://www.epa.gov/sites/default/files/2021-02/egrid2019_data.xlsx', 
                           'https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx']

load_data.download_egrid_files(egrid_files_to_download)

############# EIA-930 data #####################

load_data.download_eia930_data(years_to_download=[year])

########## Power Sector Data Crosswalk #############
# NOTE: Check for new releases at https://github.com/USEPA/camd-eia-crosswalk

load_data.download_epa_psdc(psdc_url='https://github.com/USEPA/camd-eia-crosswalk/releases/download/v0.2.1/epa_eia_crosswalk.csv')


## Load and clean the cems data

In [ ]:
## CLEAN THE EIA-923 DATA ##

pudl_out = load_data.initialize_pudl_out(year=2020)

# allocate net generation and heat input to each generator-fuel grouping
gen_fuel_allocated = allocate_gen_fuel.allocate_gen_fuel_by_generator_energy_source(pudl_out, drop_interim_cols=True)

# create a table that identifies the primary fuel of each generator and plant
primary_fuel_table = data_cleaning.create_primary_fuel_table(gen_fuel_allocated)

# calculate co2 emissions for each generator-fuel based on allocated fuel consumption
gen_fuel_allocated = data_cleaning.calculate_co2_from_fuel_consumption(gen_fuel_allocated, year)

# aggregate the allocated data to the generator level
gen_fuel_allocated = allocate_gen_fuel.agg_by_generator(gen_fuel_allocated, 
                                                        sum_cols=["net_generation_mwh", 
                                                                  "fuel_consumed_mmbtu",
                                                                  "fuel_consumed_for_electricity_mmbtu",
                                                                  "co2_mass_tons",
                                                                  "co2_mass_tons_adjusted"])

# merge the primary fuel information into the allocated data
gen_fuel_allocated = gen_fuel_allocated.merge(primary_fuel_table, how='left', on=['plant_id_eia','generator_id'])

# remove any plants that we don't want in the data
gen_fuel_allocated = data_cleaning.remove_plants(gen_fuel_allocated, 
                                                 non_grid_connected=True, 
                                                 remove_states=['PR'],
                                                 steam_only_plants=False,
                                                 distribution_connected_plants=False)

gen_fuel_allocated = data_cleaning.assign_ba_code_to_plant(gen_fuel_allocated, year)

In [ ]:

# clean the cems data
cems = data_cleaning.clean_cems(year)

# flag any generator-months for which we already have cems data
gen_fuel_allocated = data_cleaning.identify_emissions_data_source(cems, gen_fuel_allocated)

# convert hourly gross generation to net generation
cems = data_cleaning.convert_gross_to_net_generation(cems, gen_fuel_allocated)

# for generators where there is heat input but no gross generation reported, impute hourly net generation based on reported EIA values
cems = data_cleaning.impute_missing_hourly_net_generation(cems, gen_fuel_allocated)

# add information about the balancing authority 
cems = data_cleaning.assign_ba_code_to_plant(cems, year)

# for now, let's just add a placeholder column
cems['co2_mass_tons_adjusted'] = cems['co2_mass_tons']

## Output data to CSV

In [ ]:
# output the cems data to csv for others to use (update the date)
# removes any hourly data where zero data is reported in order to keep the file size smaller
cems_for_export = cems.loc[cems[['co2_mass_tons','gross_generation_mwh','fuel_consumed_mmbtu']].sum() > 0, ['plant_id_eia', 'unitid', 'operating_datetime_utc',
       'gross_generation_mwh','net_generation_mwh','steam_load_1000_lbs', 'fuel_consumed_mmbtu', 'co2_mass_tons',
       'report_date', 'cems_reporting_category', 'energy_source_code',
       'ba_code', 'state']]

date = '20220415'
cems_for_export.to_csv(f'../data/output/cems_{year}_cleaned_{date}.csv')